SVG Path instructions: https://www.w3schools.com/graphics/svg_path.asp
Bezier curves to biarcs: http://dlacko.org/blog/2016/10/19/approximating-bezier-curves-by-biarcs/

In [1]:
import re
import pandas as pd
import numpy as np
import math
import serial
import pygame
from pygame.locals import QUIT

import shapes
import plotter as plt

In [2]:
# read svg and find paths
f = open("/Users/Bas/Desktop/picassodog.svg", "r")
svg = "".join(f.readlines())
paths = re.findall(r"path\s*d=\"(.+?)\"", svg)

In [3]:
# to keep track of how many parameters an instruction needs 
num_param = {"m": 2, "l": 2, "h": 1, "v": 1, "z": 0,
            "c": 6, "s": 4, "q": 4, "t": 2, "a": 7}

In [4]:
# split svg path into its individual components
def split_path(path):
    ins = ""
    
    while len(path) > 0:
        #print path
        
        # if the first element is a letter, it is the instruction
        if path[0].isalpha():
            ins = path[0]
            path = path[1:]
            path = path.strip()
        
        # get number of necessary parameters
        n = str(num_param[ins.lower()])

        # split into part for this instruction and rest
        res = re.search(r"((?:[\-\d\.]+[\s\,]*){"+n+"})(.*)", path)
        
        # collect parameters and the remaining path
        params = res.group(1).strip()
        path = res.group(2).strip()
        
        # split parameters into individual numbers
        params = re.split(r"[,\s]", params)
        params = [float(p) for p in params]
        
        if ins == "m":
            ins = "M"
        
        # reorder the parameters
        yield [ins] + params[-2:] + params[:-2]

In [5]:
def clean_df(df):
    # flip y coordinates (we want (0,0) in the bottom left)
    df[["y", "C1y", "C2y"]] = -df[["y", "C1y", "C2y"]]

    # convert everything to absolute paths
    df["is_abs"] = df["ins"].apply(lambda x: x.isupper())
    df["group"] = df["is_abs"].cumsum()

    # add the relative paths to the absolute paths
    df["P2x"] = df.groupby("group").x.cumsum()
    df["P2y"] = df.groupby("group").y.cumsum()

    # shift y coordinate such that the origin is in (0,0)
    df.loc[df["is_abs"], "C1y"] -= df["P2y"].min()
    df.loc[df["is_abs"], "C2y"] -= df["P2y"].min()
    df["P2y"] -= df["P2y"].min()

    # add 'from' coordinates such that every row in the DataFrame is stand-alone
    df["P1x"] = df["P2x"].shift(1)
    df["P1y"] = df["P2y"].shift(1)
    df.loc[0, "P1x"] = 0
    df.loc[0, "P1y"] = 0

    # make these absolute as well
    df.loc[-df["is_abs"], "C1x"] += df["P1x"]
    df.loc[-df["is_abs"], "C1y"] += df["P1y"]
    df.loc[-df["is_abs"], "C2x"] += df["P1x"]
    df.loc[-df["is_abs"], "C2y"] += df["P1y"]

    # make sure that these are in uppercase
    df["ins"] = df["ins"].str.upper()

    df = df[["ins", "P1x", "P1y", "P2x", "P2y", "C1x", "C1y", "C2x", "C2y"]]
    
    return df

In [6]:
def normalize_df(df, xlim = 100., ylim = 100., margin = 5.):
    # make sure that x and y are between 0 and xlim, ylim
    # with a margin
    # make sure everything is larger than 0
    xmin = df[["P1x", "P2x", "C1x", "C2x"]].min().min()
    df[["P1x", "P2x", "C1x", "C2x"]] -= xmin

    ymin = df[["P1y", "P2y", "C1y", "C2y"]].min().min()
    df[["P1y", "P2y", "C1y", "C2y"]] -= ymin

    # make sure everything is less than xlim - 2*margin
    xmax = df[["P1x", "P2x", "C1x", "C2x"]].max().max()
    xmul = (xlim-2*margin) / xmax

    ymax = df[["P1y", "P2y", "C1y", "C2y"]].max().max()
    ymul = (ylim-2*margin) / ymax

    df[["P1x", "P1y", "P2x", "P2y", "C1x", "C1y", "C2x", "C2y"]] *= min(xmul, ymul)

    # shift everything up by margin
    df[["P1x", "P1y", "P2x", "P2y", "C1x", "C1y", "C2x", "C2y"]] += margin

    return df

In [7]:
def preview_objs(objs):
    pygame.init()

    canvas=pygame.display.set_mode((240,170),0,32)

    white = (255, 255, 255)
    black = (0, 0, 0)

    canvas.fill(white)

    for obj in objs:
        obj.draw(canvas, black)

    while True:
        for event in pygame.event.get():
            if event.type==QUIT:
                pygame.quit()
                return
        pygame.display.update()

In [8]:
def path_to_df(path):
    records = split_path(path)
    cols = ["ins", "x", "y", "C1x", "C1y", "C2x", "C2y"]
    df = pd.DataFrame.from_records(records, columns = cols)
    
    df = clean_df(df)
    
    df = normalize_df(df)
    
    return df

In [9]:
def objs_to_plotter(objs):
    for obj in objs:
        print(obj.plot_instructions())

In [10]:
def df_to_plotter(df):
    objs = df_to_objs(df)
    preview_objs(objs)
    res = input("Are you sure?")
    if res == "y":
        objs_to_plotter(objs)
    else:
        return

In [11]:
def df_to_objs(df):
    objs = []

    for i, row in df.iterrows():
        if row["ins"] == "M":
            P2 = shapes.Point(*row[["P2x", "P2y"]])
            objs.append(P2)
            
        elif row["ins"] == "C":
            P1 = shapes.Point(*row[["P1x", "P1y"]])
            P2 = shapes.Point(*row[["P2x", "P2y"]])
            C1 = shapes.Point(*row[["C1x", "C1y"]])
            C2 = shapes.Point(*row[["C2x", "C2y"]])
            bez = shapes.CubicBezier(P1, C1, C2, P2)

            # split curve if there is an inflection point
            ip1, ip2 = bez.inflection_points()

            if (ip1 == -1) and (ip2 != -1): # they are ordered
                bez1, bez2 = bez.split_at(ip2)
                objs.append(bez2)
                objs.append(bez1)
            elif (ip1 != -1) and (ip2 != -1): # split twice
                bez1, bez2 = bez.split_at(ip1)
                ip2 = (1 - ip1) * ip2
                bez3, bez4 = bez2.split_at(ip2)
                objs.append(bez4)
                objs.append(bez3)
                objs.append(bez1)
            else:
                objs.append(bez)
            
    return objs

In [12]:
# loop over paths in svg
for path in paths:
    df = path_to_df(path)

In [13]:
df_to_plotter(df)

KeyboardInterrupt: 